In [3]:
import sys
import jsonlines


# keys -> id, input, output
train_dict = {'id': [], 'input': [], 'output': []}
with jsonlines.open("fever-train-kilt.jsonl") as f:
    for line in f.iter():
        train_dict['id'].append(line['id'])
        train_dict['input'].append(line['input'])
        train_dict['output'].append(line['output'])

In [4]:
len(train_dict['id'])

104966

In [7]:
list(train_dict['input'])[:10]

['Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.',
 'Roman Atwood is a content creator.',
 'History of art includes architecture, dance, sculpture, music, painting, poetry literature, theatre, narrative, film, photography and graphic arts.',
 'Adrienne Bailon is an accountant.',
 'Homeland is an American television spy thriller based on the Israeli television series Prisoners of War.',
 'The Boston Celtics play their home games at TD Garden.',
 'The Ten Commandments is an epic film.',
 'Tetris has sold millions of physical copies.',
 'Cyndi Lauper won the Best New Artist award at the 27th Grammy Awards in 1985.',
 'There is a movie called The Hunger Games.']

In [11]:
list(train_dict['output'])[:3]

[[{'answer': 'SUPPORTS'}],
 [{'answer': 'SUPPORTS',
   'provenance': [{'wikipedia_id': '43386202',
     'title': 'Roman Atwood',
     'start_paragraph_id': 1,
     'start_character': 111,
     'end_paragraph_id': 1,
     'end_character': 181,
     'bleu_score': 0.7404008324993688,
     'section': 'Section::::Abstract.'}]},
  {'answer': 'SUPPORTS'}],
 [{'answer': 'SUPPORTS'}]]

In [16]:
wiki_dict = {'input': [], 'wiki_id': [], 'title': []}

for _input, _output in zip(train_dict['input'], train_dict['output']):
    
    title_list = []; id_list = []
    for elem in _output:
        if 'provenance' in elem.keys():
            for _elem in elem['provenance']:
                title_list.append(_elem['title'])
                id_list.append(_elem['wikipedia_id'])
    #if len(set(title_list)) > 1: print(title_list)
    wiki_dict['input'].append(_input)
    wiki_dict['wiki_id'].append(id_list)
    wiki_dict['title'].append(title_list)

In [17]:
len(wiki_dict['input'])

104966

In [18]:
train_dict = {'input': [], 'wiki_id': [], 'title': []}
dev_dict = {'input': [], 'wiki_id': [], 'title': []}
test_dict = {'input': [], 'wiki_id': [], 'title': []}

for i, (_input, _id, _title) in enumerate(zip(wiki_dict['input'], wiki_dict['wiki_id'], wiki_dict['title'])):
    if i<60000:
        train_dict['input'].append(_input)
        train_dict['wiki_id'].append(_id)
        train_dict['title'].append(_title)
    elif i<60000+7000:
        dev_dict['input'].append(_input)
        dev_dict['wiki_id'].append(_id)
        dev_dict['title'].append(_title)
    elif i<60000+7000+10000:
        test_dict['input'].append(_input)
        test_dict['wiki_id'].append(_id)
        test_dict['title'].append(_title)
    else:
        continue


In [19]:
print(f'train: {len(train_dict["input"])}\ndev: {len(dev_dict["input"])}\ntest: {len(test_dict["input"])}')

train: 60000
dev: 7000
test: 10000


In [28]:
def dict2csv(_dict):
    print(f'Dict #: {len(_dict["input"])}')
    df = {'input': [], 'output': []}
    for _input, _title_list in zip(_dict['input'], _dict['title']):
        for _title in _title_list:
            df['input'].append(_input)
            df['output'].append(_title)
    print(f'Csv #: {len(df["input"])}')
    print(' ')
    return df
            
train_csv = dict2csv(train_dict)
dev_csv = dict2csv(dev_dict)
test_csv = dict2csv(test_dict)

Dict #: 60000
Csv #: 65231
 
Dict #: 7000
Csv #: 7544
 
Dict #: 10000
Csv #: 10770
 


In [29]:
import pickle
import pandas as pd

with open('train_total.pickle', 'wb') as f:
    pickle.dump(wiki_dict, f)
    
df = pd.DataFrame(train_csv)
df.to_csv('fever_toy_train.csv')
df = pd.DataFrame(dev_csv)
df.to_csv('fever_toy_dev.csv')
df = pd.DataFrame(test_csv)
df.to_csv('fever_toy_test.csv')

In [31]:
total_title = list(set(train_csv['output'] + dev_csv['output'] + test_csv['output']))
len(total_title)

6063

In [32]:
df = pd.DataFrame({'corpus': total_title})
df.to_csv('scale_1_kilt_corpus.csv')

In [33]:
def csv2pickle(_csv):
    _pickle = {'input': list(_csv['input']), 'output': list(_csv['output'])}
    return _pickle

train_p = csv2pickle(train_csv)
dev_p = csv2pickle(dev_csv)
test_p = csv2pickle(test_csv)

with open('fever_toy_train.pickle', "wb") as f:
    pickle.dump(train_p, f)
with open('fever_toy_dev.pickle', 'wb') as f:
    pickle.dump(dev_p, f)
with open('fever_toy_test.pickle', 'wb') as f:
    pickle.dump(test_p, f)